# Finance Dashboard 

## 0. Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn
# import streamlit as st

In [2]:
import FinanceDataReader as fdr
import quantstats as qs

In [11]:
## custom libs

from pricevolume.loader import KRXPriceDM
from pricevolume.utils import DateUtil

## 1. Import data

- Import price-volume data from KRX using custom lib `financedashboard.pricevolume`
- Import supplement return & adj.close data from naver (via `FinanceDataReader`)

In [6]:
# Global parameters

START = 20140101
END = 20220520

In [7]:
pricevolume = KRXPriceDM(START, END)

In [9]:
# Download price-volume data from KRX.
# Also, build cache for faster access. (Both lv1 and lv2)

close_df = pricevolume.get_data("close")

In [10]:
close_df

ISU_SRT_CD,000020,000040,000050,000060,000070,000075,000080,000087,000100,000105,...,37550L,388050,389140,405640,412930,413600,415580,419270,389260,399720
trdDd,,,,,,,,,,,,,,,,,,,,,
2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-02,4440,438,111500,15500,73500,32600,21750,16850,182500,93300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,4540,440,114000,15400,71800,33400,21450,16850,180500,93000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-20,11850,780,15050,39100,80800,62800,35550,20900,58500,56900,...,38200,22900,27000,2185,2195,2445,2165,2215,20100,27100
2022-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Create additional DMs

- tradingday DM by processing `pricevolume`
- return & adj.close DM by mixing `pricevolume` and `FinanceDataReader`

### Download additional data for new DMs

- KRX sector DM downloaded from KRX
- Corporate finance fundamental analysis data from DART (using 3rd party libraries)

## 3. Make price-volume chart graph

- Simple price-volume chart using both `seaborn`/`streamlit`
- Show biggest winners/losers on mouse hover
    - Make pseudo-KOSPI portfolio mimicking KOSPI index
    - Show Top 3 winners/losers on the graph 

## 4. Make Finviz-style stock tree map

## 5. Make return correlation clustering DM 

- Also, visualize the result

## 6. Create NLP news DM